<h1>THIS NOTEBOOKS ATTEMPTS TO FIND ASSOCIATIONS AMONG THE TECHNOLOGIES USED BY THE RESPONDENTS OF THE KAGGLE SURVEY 2020 

In [ ]:
import warnings
warnings.filterwarnings("ignore")
!pip -q install apyori
from termcolor import colored
import numpy as np
import pandas as pd
from apyori import apriori

data = pd.read_csv("../input/kaggle-survey-2020/kaggle_survey_2020_responses.csv",header=1)

In [ ]:
#Excluding Columns having 'None' and 'Other' as responses. Also excluding questions about social media, courses and future plans.
product_data = data.loc[:, pd.Series(data.columns)[(pd.Series(data.columns).str.contains('- Selected Choice -',regex=False)) & \
                                         (pd.Series(data.columns).str.contains('(None$)|(Other$)|activities|courses|(media sources)|(you hope)',regex=True)==False)]].copy()

In [ ]:
#Identifying questions having same responses
dupes = pd.DataFrame()
for i in product_data.columns:
    dupes = dupes.append(pd.DataFrame({"Duplicate":pd.Series(product_data.loc[:,i].dropna().unique()).apply(lambda x: x.strip()),"Column Name":i}))
    
dupes[dupes.duplicated(subset="Duplicate")]

MATLAB and Shiny are present in multiple questions

In [ ]:
product_data.columns = range(len(product_data.columns))

In [ ]:
dupes = pd.DataFrame()
for i in product_data.columns:
    dupes = dupes.append(pd.DataFrame({"Duplicate":pd.Series(product_data.loc[:,i].dropna().unique()).apply(lambda x: x.strip()),"Column Name":i}))
    
dupes[dupes.duplicated(subset="Duplicate")]

In [ ]:
product_data.iloc[:,20] = product_data.iloc[:,20].str.replace("MATLAB","MATLAB IDE",regex=False)
product_data.iloc[:,168] = product_data.iloc[:,168].str.replace("Shiny","Shiny (Publicly share)",regex=False)
product_data.fillna('',inplace=True)

In [ ]:
#first 5 records
product_data.head()

In [ ]:
# Converting the data frame of transcations to list of lists of transcations
product_list = list()
for t in range(len(product_data)):
    product_list.append([str(product_data.values[t,c]) for c in range(product_data.shape[1]) if str(product_data.values[t,c])!=''])

In [ ]:
len(product_list)

In [ ]:
print(product_list[0])

In [ ]:
MIN_SUPPORT = 0.02
MIN_CONFIDENCE = 0.6
MIN_LIFT = 3
MAX_LENGTH = 2

rules = apriori(product_list,min_support=MIN_SUPPORT,min_confidence=MIN_CONFIDENCE,min_lift=MIN_LIFT,max_length=MAX_LENGTH)
rules = list(rules)

rules_df = pd.DataFrame()
for i in range(len(rules)):
    rules_df = rules_df.append(pd.DataFrame({"Antecedent": (list(rules[i][2][0][0])[0]).strip(),
                               "Consequent": (list(rules[i][2][0][1])[0]).strip(),
                               "Support": np.round(float(list(rules[i])[1]),3),
                               "Confidence": np.round(float(list(rules[i])[2][0][2]),3),
                               "Lift": np.round(float(list(rules[i])[2][0][3]),3)},index=[i]))
    
    
rules_df = rules_df.loc[(rules_df["Consequent"]!='') & (rules_df["Antecedent"]!='')].sort_values(by=["Lift","Confidence"],ascending=[False,False]).drop_duplicates().reset_index(drop=True).copy()

rules_df.head(10)

<h1>Findings with support > 0.02, confidence > 0.7, and lift > 3</h1>

In [ ]:
rules_df.sort_values(by='Confidence',ascending=False,inplace = True)
n = 1
for i in range(len(rules_df)):
    if (rules_df.iloc[i,3]*100) < 70:
        break
    print(f'{n}. {colored(np.round(rules_df.iloc[i,3]*100,1),"grey","on_cyan")}{colored("%","grey","on_cyan")} of the respondents who use {colored(rules_df.iloc[i,0],"grey","on_cyan")} also use {colored(rules_df.iloc[i,1],"grey","on_cyan")}\n')
    n += 1

<h2>There are a few interesting findings like:</h2>
    <h3><ol><li>78.6 % of Respondents having personal blog also use Github (might be to host their code snippets).</li>
    <li>78.2% of Respondents using Kaggle also use Github.</li>

<h1>Let's look at the top 20 technologies used by the respondents

In [ ]:
prod_arr = pd.Series(product_data.values.flatten())
prod_arr = prod_arr[prod_arr != ''].copy()
(prod_arr.value_counts()).iloc[:20].plot(kind='bar',figsize=(15,5),title="Top 20 Technologies Used",color='black');